In [ ]:
import pandas as pd
import mysql.connector

def insert_data_into_table(curr, conn, table_name, data, query):
    try:
        for i, row in data.iterrows():
            curr.execute(query, list(row))
        conn.commit()
        print(f"Data inserted into {table_name} table successfully.")
    except Exception as e:
        print(f"Error inserting into {table_name} table: {e}")

def main():
    # Load data
    customer_data = pd.read_json("customers.json")
    transaction_data = pd.read_json("transaction_logs.json")

    # Connect to the database
    conn = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="1234",
        database="projectp0"
    )
    curr = conn.cursor()

    # Insert data into Customer table
    customer_insert_query = """
    INSERT INTO Customer (customer_id, customer_name, country, city)
    VALUES (%s, %s, %s, %s);
    """
    insert_data_into_table(curr, conn, "Customer", customer_data, customer_insert_query)

    # Insert data into Product table
    product_insert_query = """
    INSERT INTO Product (product_id, product_name, product_category, price, ecommerce_website_name)
    VALUES (%s, %s, %s, %s, %s);
    """
    product_table = transaction_data[["product_id", "product_name", "product_category", "price", "ecommerce_website_name"]].drop_duplicates(subset=['product_id'])
    insert_data_into_table(curr, conn, "Product", product_table, product_insert_query)

    # Insert data into Orders table
    order_insert_query = """
    INSERT INTO Orders (order_id, customer_id, product_id, qty, datetime)
    VALUES (%s, %s, %s, %s, %s);
    """
    orders_table = transaction_data[["order_id", "customer_id", "product_id", "qty", "datetime"]]
    insert_data_into_table(curr, conn, "Orders", orders_table, order_insert_query)

    # Insert data into Payment table
    payment_insert_query = """
    INSERT INTO Payment (payment_txn_id, order_id, payment_type, payment_txn_success, failure_reason)
    VALUES (%s, %s, %s, %s, %s);
    """
    payment_table = transaction_data[["payment_txn_id", "order_id", "payment_type", "payment_txn_success", "failure_reason"]]
    insert_data_into_table(curr, conn, "Payment", payment_table, payment_insert_query)

    # Close connection
    curr.close()
    conn.close()
    print("Database connection closed.")

if __name__ == "__main__":
    main()
